# Pytorch Optuna

In [1]:
!pip install pytorch-lightning optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
import optuna
from pytorch_lightning.loggers import TensorBoardLogger

In [3]:
transform = transforms.ToTensor()
train_dataset = MNIST(root=".", train=True, download=True, transform=transform)
test_dataset = MNIST(root=".", train=False, download=True, transform=transform)

train_len = int(0.8 * len(train_dataset))
val_len = len(train_dataset) - train_len
train_data, val_data = random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train_data, batch_size=64)
val_loader = DataLoader(val_data, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

100%|██████████| 9.91M/9.91M [00:00<00:00, 45.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.64MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.6MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.35MB/s]


In [ ]:
class LitModel(pl.LightningModule):
  def __init__(self, lr=1e-3, dropout=0.3):
    super().__init__()
    self.save_hyperparameters()
    self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 10)
        )

  def forward(self, x):
      return self.model(x)

  def training_step(self, batch, batch_idx):
      x, y = batch
      logits = self(x)
      loss = F.cross_entropy(logits, y)
      acc = (logits.argmax(dim=1) == y).float().mean()
      self.log("train_loss", loss, prog_bar=True)
      self.log("train_acc", acc, prog_bar=True)
      return loss

  def validation_step(self, batch, batch_idx):
      x, y = batch
      logits = self(x)
      loss = F.cross_entropy(logits, y)
      acc = (logits.argmax(dim=1) == y).float().mean()
      self.log("val_loss", loss, prog_bar=True)
      self.log("val_acc", acc, prog_bar=True)
      return loss

  def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

def objective(trial):
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    model = LitModel(lr=lr, dropout=dropout)

    trainer = pl.Trainer(
        max_epochs=5,
        logger=False,
        enable_progress_bar=False,
        accelerator="auto",
        devices=1,
        num_sanity_val_steps=0,
        enable_model_summary=False
    )

    trainer.fit(model, train_loader, val_loader)
    val_loss = trainer.callback_metrics["val_loss"].item()
    return val_loss

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("✅ Best hyperparameters:", study.best_trial.params)

best_params = study.best_trial.params
final_model = LitModel(**best_params)

trainer = pl.Trainer(
    max_epochs=5,
    accelerator="auto",
    devices=1,
    precision=16
)

trainer.fit(final_model, train_loader, val_loader)
trainer.test(final_model, test_loader)

[I 2025-04-24 17:42:46,161] A new study created in memory with name: no-name-d583b782-17d8-4d74-8c44-054f446f9f27
INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.
[I 2025-04-24 17:43:58,317] Trial 0 finished with value: 0.2424115538597107 and parameters: {'lr': 0.02159934329729097, 'dropout': 0.19415315790914833}. Best is trial 0 with value: 0.2424115538597107.
INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INF